In [4]:
!pip install pytorch-gradcam

  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-py3-none-any.whl size=5269 sha256=1ed491720611f7e64ebc28318a382102e6f0e3f0c4b4675528b0e94593774b5a
  Stored in directory: c:\users\bruno\appdata\local\pip\cache\wheels\dd\3c\0e\8b7512f5d7f462c9c37342107b7f0276609345f246ae4e75fc
Successfully built pytorch-gradcam


In [14]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch
import cv2
import numpy as np
import requests
import torchvision.transforms as transforms
#from gradcam import EigenCAM
from gradcam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image
import sys
from torchvision.io import read_image


sys.path.insert(0, '../../scripts/')
from nets.baselinenet import BaselineNetReg, BaselineNetNoReg
from nets.lenet import LeNet
from monai.networks.nets import DenseNet121
from helpers import preprocessing as prep
from helpers import miscellaneous as misc

CONFIG = misc.get_config()
COLORS = np.random.uniform(0, 255, size=(80, 3))

ModuleNotFoundError: No module named 'gradcam.utils.image'; 'gradcam.utils' is not a package

In [ ]:
img_path = '../../data/raw/flattened/AprilFx16_dp.jpg'

img = read_image(img_path).float()

transformer, _  = prep.get_transformer('MonaiDense', img_resize=32)

model_path = '../../models/DenseNet121_LR0.0001_CRCross Entropy_TRMonaiDense_OPAdam_BS8_EP20__ID_e0e4e329-fa9d-4d7b-8cf1-9a969f25b197.pth'
model_checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=2).eval()
model.load_state_dict(model_checkpoint)

In [ ]:
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)

transformer = transforms.Compose(
    [
        ScaleIntensity(),
        EnsureType(),
        transforms.Grayscale(),
        transforms.Resize((CONFIG['IMAGE_RESIZE'],CONFIG['IMAGE_RESIZE'])),
    ]
)


In [16]:
!pip install torchcam

In [17]:
from torchcam.methods import SmoothGradCAMpp

cam_extractor = SmoothGradCAMpp(model, target_layer=[model.features[-1]])
# Get your input
# Preprocess it for your chosen model+
input_tensor = transformer(img)
input_tensor = input_tensor.unsqueeze(dim=0)
# Preprocess your data and feed it to the model
out = model(input_tensor)
# Retrieve the CAM by passing the class index and the model output
activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

NameError: name 'model' is not defined

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask

result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
# Display it
plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

In [ ]:
img.size()

In [ ]:
def grad_cam(model, trained_model_path, transformer_name, target_layer, image):
    CONFIG = misc.get_config()

    transformer, _  = prep.get_transformer(transformer_name)

    model_checkpoint = torch.load(trained_model_path, map_location=torch.device('cpu'))

    model = model.eval()
    model.load_state_dict(model_checkpoint)

    cam_extractor = SmoothGradCAMpp(model, target_layer=target_layer)

    input_tensor = transformer(image)
    input_tensor = input_tensor.unsqueeze(dim=0)

    out = model(input_tensor)

    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

    result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
    plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

grad_cam(model, model_path, 'MonaiDense', [model.features[-1]], img)